In [3]:
import rospy
from thr_infrastructure_msgs.srv import GetCarryPose, GetCarryPoseRequest
import tf
import os
import transformations

In [4]:
rospy.init_node('create_grid')

In [3]:
srv = rospy.ServiceProxy('thr/get_carry_pose', GetCarryPose)
rospy.wait_for_service('thr/get_carry_pose')
tfl = tf.TransformListener()

In [5]:
ObjectTGripper = tfl.lookupTransform('shapeo', 'right_gripper', rospy.Time(0))

In [6]:
ObjectTGripper

((-0.02861536745927118, 0.01184654044929978, 0.0676455700946112),
 (0.6943042452547984,
  0.6234654355113374,
  -0.03133961024470876,
  0.3581205028667489))

## Defintion of functions

In [7]:
def get_handover_location(method):
    success = False
    while not success:
        rospy.sleep(0.1)
        req = GetCarryPoseRequest()
        req.object = '/shapeo'
        req.location = 'star'
        req.method = method
        res = srv(req)
        success = res.success
    # express the result in VREP frame
    VrepTBase = tfl.lookupTransform('vrep_frame', 'base', rospy.Time(0))
    GripperTVrep = tfl.lookupTransform('right_gripper', 'vrep_tip', rospy.Time(0))
    BaseTObject = transformations.pose_to_list(res.object_pose)
    # apply chain rule of transformation
    robot_pose = transformations.multiply_transform(VrepTBase, BaseTObject)
    robot_pose = transformations.multiply_transform(robot_pose, ObjectTGripper)
    robot_pose = transformations.multiply_transform(robot_pose, GripperTVrep)
    return robot_pose, BaseTObject

In [8]:
def convert_to_string(pose):
    # create the string of the pose to write in the textfile
    pose_str = ''
    for lists in pose:
        for value in lists:
            pose_str += str(value) + '\t' 
    pose_str += '\n'
    return pose_str

## Create table of hip positions 

In [17]:
grid_corners = []
hip_pose = []
object_pose = []
# get the 4 corners of the grid by asking the human to move between them
for i in range(4):
    rospy.sleep(5)
    vrep, obj = get_handover_location('reba')
    grid_corners.append(vrep)
    object_pose.append(obj)
    vrep, obj = get_handover_location('relative')
    grid_corners.append(vrep)
    object_pose.append(obj)
    # get hip pose for checking
    hip_pose.append(tfl.lookupTransform('base', '/human/tracker/base', rospy.Time(0)))
    os.system('beep')
    rospy.sleep(0.5)
    os.system('beep')

## Pulbish them to check result in Rviz

In [18]:
tfb = tf.TransformBroadcaster()

In [19]:
rate = rospy.Rate(10)
while not rospy.is_shutdown():
    for i in range(len(grid_corners)):
        tfb.sendTransform(grid_corners[i][0], grid_corners[i][1], rospy.Time.now(),
                          '/vrep/corner/'+str(i), '/vrep_frame')
        tfb.sendTransform(object_pose[i][0], object_pose[i][1], rospy.Time.now(),
                          '/object/'+str(i), 'base')
    for i in range(len(hip_pose)):
        tfb.sendTransform(hip_pose[i][0], hip_pose[i][1], rospy.Time.now(),
                          '/human/hip/'+str(i), 'base')
    rate.sleep()

KeyboardInterrupt: 

## Write result to a text file

In [23]:
pose_reba = ''
pose_naive = ''
for i in range(4):
    pose_reba += convert_to_string(grid_corners[i])
    pose_naive += convert_to_string(grid_corners[i+1])
# write the string to the file
with open('/tmp/grid_reba.txt', 'w') as f:
    f.write(pose_reba)
with open('/tmp/grid_naive.txt', 'w') as f:
    f.write(pose_naive)